In [0]:
import pandas as pd
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',500)
import numpy as np
from sklearn import preprocessing 
import matplotlib.pyplot as plt
plt.rc('font',size=14)
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style='white')
sns.set(style='whitegrid',color_codes=True)
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier

In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df=pd.read_csv('Churn.csv')

In [65]:
df.head()

customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService     MultipleLines InternetService OnlineSecurity OnlineBackup DeviceProtection TechSupport StreamingTV StreamingMovies        Contract PaperlessBilling              PaymentMethod  MonthlyCharges TotalCharges Churn
0  7590-VHVEG  Female              0     Yes         No       1           No  No phone service             DSL             No          Yes               No          No          No              No  Month-to-month              Yes           Electronic check           29.85        29.85    No
1  5575-GNVDE    Male              0      No         No      34          Yes                No             DSL            Yes           No              Yes          No          No              No        One year               No               Mailed check           56.95       1889.5    No
2  3668-QPYBK    Male              0      No         No       2          Yes                No             DSL            Yes          Yes               No          No          No              No  Month-to-month              Yes               Mailed check           53.85       108.15   Yes
3  7795-CFOCW    Male              0      No         No      45           No  No phone service             DSL            Yes           No              Yes         Yes          No              No        One year               No  Bank transfer (automatic)           42.30      1840.75    No
4  9237-HQITU  Female              0      No         No       2          Yes                No     Fiber optic             No           No               No          No          No              No  Month-to-month              Yes           Electronic check           70.70       151.65   Yes

In [66]:
print(df['MultipleLines'].value_counts())
print(df['InternetService'].value_counts())
print(df['Contract'].value_counts())

No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64
Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64
Month-to-month    3875
Two year          1695
One year          1473
Name: Contract, dtype: int64


In [67]:
missing_df=pd.DataFrame()
missing_df['cols']=df.isnull().sum().keys()
missing_df['values']=df.isnull().sum().values.tolist()
missing_df[missing_df['values']>0]

Empty DataFrame
Columns: [cols, values]
Index: []

In [0]:
df['Churn'].replace('Yes',1,inplace=True)
df['Churn'].replace('No',0,inplace=True)

In [0]:
#mapping is performed to convert string into numeric values
df['Partner']=df['Partner'].map({'Yes': 1, 'No': 0})
df['Dependents']=df['Dependents'].map({'Yes': 1, 'No': 0})
df['PhoneService']=df['PhoneService'].map({'Yes': 1, 'No': 0})
df['OnlineBackup']=df['OnlineBackup'].map({'Yes': 1, 'No': 0,'No internet service': 0})
df['DeviceProtection']=df['DeviceProtection'].map({'Yes': 1, 'No': 0,'No internet service': 0})
df['TechSupport']=df['TechSupport'].map({'Yes': 1, 'No': 0,'No internet service': 0})
df['StreamingMovies']=df['StreamingMovies'].map({'Yes': 1, 'No': 0,'No internet service': 0})
df['StreamingTV']=df['StreamingTV'].map({'Yes': 1, 'No': 0,'No internet service': 0})
df['OnlineSecurity']=df['OnlineSecurity'].map({'Yes': 1, 'No': 0,'No internet service': 0})
df['MultipleLines']=df['MultipleLines'].map({'Yes': 1, 'No': 0,'No phone service': 0})

In [0]:
#Features are made based on agregation
df['avg_monthly_charge'] = df.groupby('gender')['MonthlyCharges'].transform('mean')
df['avg_monthly_charge_P'] = df.groupby('Partner')['MonthlyCharges'].transform('mean')
df['avg_monthly_charge_c'] = df.groupby('Contract')['MonthlyCharges'].transform('mean')
df['avg_monthly_charge_d'] = df.groupby('gender')['Dependents'].transform('mean')
df['avg_monthly_charge_t'] = df.groupby('tenure')['MonthlyCharges'].transform('mean')
df['avg_monthly_charge_sc'] = df.groupby('SeniorCitizen')['MonthlyCharges'].transform('mean')
df['avg_monthly_charge_is'] = df.groupby('InternetService')['MonthlyCharges'].transform('mean')
df['avg_monthly_charge_pm'] = df.groupby('PaymentMethod')['MonthlyCharges'].transform('mean')

In [0]:
#Dummy variables are made
#dummy1=pd.get_dummies(df['MultipleLines'])
dummy2=pd.get_dummies(df['InternetService'])
dummy3=pd.get_dummies(df['Contract'])
dummy4=pd.get_dummies(df['PaymentMethod'])

In [72]:
df=pd.concat([df,dummy2,dummy3,dummy4],axis=1)
df.head()

customerID  gender  SeniorCitizen  Partner  Dependents  tenure  PhoneService  MultipleLines InternetService  OnlineSecurity  OnlineBackup  DeviceProtection  TechSupport  StreamingTV  StreamingMovies        Contract PaperlessBilling              PaymentMethod  MonthlyCharges TotalCharges  Churn  avg_monthly_charge  avg_monthly_charge_P  avg_monthly_charge_c  avg_monthly_charge_d  avg_monthly_charge_t  avg_monthly_charge_sc  avg_monthly_charge_is  avg_monthly_charge_pm  DSL  Fiber optic  No  \
0  7590-VHVEG  Female              0        1           0       1             0              0             DSL               0             1                 0            0            0                0  Month-to-month              Yes           Electronic check           29.85        29.85      0           65.204243             67.776264             66.398490              0.294725             50.485808              61.847441              58.102169              76.255814    1            0   0   
1  5575-GNVDE    Male              0        0           0      34             1              0             DSL               1             0                 1            0            0                0        One year               No               Mailed check           56.95       1889.5      0           64.327482             61.945001             65.048608              0.304360             69.644615              61.847441              58.102169              43.917060    1            0   0   
2  3668-QPYBK    Male              0        0           0       2             1              0             DSL               1             1                 0            0            0                0  Month-to-month              Yes               Mailed check           53.85       108.15      1           64.327482             61.945001             66.398490              0.304360             57.206303              61.847441              58.102169              43.917060    1            0   0   
3  7795-CFOCW    Male              0        0           0      45             0              0             DSL               1             0                 1            1            0                0        One year               No  Bank transfer (automatic)           42.30      1840.75      0           64.327482             61.945001             65.048608              0.304360             71.245902              61.847441              58.102169              67.192649    1            0   0   
4  9237-HQITU  Female              0        0           0       2             1              0     Fiber optic               0             0                 0            0            0                0  Month-to-month              Yes           Electronic check           70.70       151.65      1           65.204243             61.945001             66.398490              0.294725             57.206303              61.847441              91.500129              76.255814    0            1   0   

   Month-to-month  One year  Two year  Bank transfer (automatic)  Credit card (automatic)  Electronic check  Mailed check  
0               1         0         0                          0                        0                 1             0  
1               0         1         0                          0                        0                 0             1  
2               1         0         0                          0                        0                 0             1  
3               0         1         0                          1                        0                 0             0  
4               1         0         0                          0                        0                 1             0

In [0]:
df['OnlineSecurity'].replace('Yes',1,inplace=True)
df['OnlineSecurity'].replace('No',0,inplace=True)

In [74]:
df.head()

customerID  gender  SeniorCitizen  Partner  Dependents  tenure  PhoneService  MultipleLines InternetService  OnlineSecurity  OnlineBackup  DeviceProtection  TechSupport  StreamingTV  StreamingMovies        Contract PaperlessBilling              PaymentMethod  MonthlyCharges TotalCharges  Churn  avg_monthly_charge  avg_monthly_charge_P  avg_monthly_charge_c  avg_monthly_charge_d  avg_monthly_charge_t  avg_monthly_charge_sc  avg_monthly_charge_is  avg_monthly_charge_pm  DSL  Fiber optic  No  \
0  7590-VHVEG  Female              0        1           0       1             0              0             DSL               0             1                 0            0            0                0  Month-to-month              Yes           Electronic check           29.85        29.85      0           65.204243             67.776264             66.398490              0.294725             50.485808              61.847441              58.102169              76.255814    1            0   0   
1  5575-GNVDE    Male              0        0           0      34             1              0             DSL               1             0                 1            0            0                0        One year               No               Mailed check           56.95       1889.5      0           64.327482             61.945001             65.048608              0.304360             69.644615              61.847441              58.102169              43.917060    1            0   0   
2  3668-QPYBK    Male              0        0           0       2             1              0             DSL               1             1                 0            0            0                0  Month-to-month              Yes               Mailed check           53.85       108.15      1           64.327482             61.945001             66.398490              0.304360             57.206303              61.847441              58.102169              43.917060    1            0   0   
3  7795-CFOCW    Male              0        0           0      45             0              0             DSL               1             0                 1            1            0                0        One year               No  Bank transfer (automatic)           42.30      1840.75      0           64.327482             61.945001             65.048608              0.304360             71.245902              61.847441              58.102169              67.192649    1            0   0   
4  9237-HQITU  Female              0        0           0       2             1              0     Fiber optic               0             0                 0            0            0                0  Month-to-month              Yes           Electronic check           70.70       151.65      1           65.204243             61.945001             66.398490              0.294725             57.206303              61.847441              91.500129              76.255814    0            1   0   

   Month-to-month  One year  Two year  Bank transfer (automatic)  Credit card (automatic)  Electronic check  Mailed check  
0               1         0         0                          0                        0                 1             0  
1               0         1         0                          0                        0                 0             1  
2               1         0         0                          0                        0                 0             1  
3               0         1         0                          1                        0                 0             0  
4               1         0         0                          0                        0                 1             0

In [0]:
df=df.fillna(0)

In [0]:
df=df.drop(['InternetService','Contract','PaymentMethod','TotalCharges','PaperlessBilling'],axis=1)

In [77]:
df.head()

customerID  gender  SeniorCitizen  Partner  Dependents  tenure  PhoneService  MultipleLines  OnlineSecurity  OnlineBackup  DeviceProtection  TechSupport  StreamingTV  StreamingMovies  MonthlyCharges  Churn  avg_monthly_charge  avg_monthly_charge_P  avg_monthly_charge_c  avg_monthly_charge_d  avg_monthly_charge_t  avg_monthly_charge_sc  avg_monthly_charge_is  avg_monthly_charge_pm  DSL  Fiber optic  No  Month-to-month  One year  Two year  Bank transfer (automatic)  Credit card (automatic)  \
0  7590-VHVEG  Female              0        1           0       1             0              0               0             1                 0            0            0                0           29.85      0           65.204243             67.776264             66.398490              0.294725             50.485808              61.847441              58.102169              76.255814    1            0   0               1         0         0                          0                        0   
1  5575-GNVDE    Male              0        0           0      34             1              0               1             0                 1            0            0                0           56.95      0           64.327482             61.945001             65.048608              0.304360             69.644615              61.847441              58.102169              43.917060    1            0   0               0         1         0                          0                        0   
2  3668-QPYBK    Male              0        0           0       2             1              0               1             1                 0            0            0                0           53.85      1           64.327482             61.945001             66.398490              0.304360             57.206303              61.847441              58.102169              43.917060    1            0   0               1         0         0                          0                        0   
3  7795-CFOCW    Male              0        0           0      45             0              0               1             0                 1            1            0                0           42.30      0           64.327482             61.945001             65.048608              0.304360             71.245902              61.847441              58.102169              67.192649    1            0   0               0         1         0                          1                        0   
4  9237-HQITU  Female              0        0           0       2             1              0               0             0                 0            0            0                0           70.70      1           65.204243             61.945001             66.398490              0.294725             57.206303              61.847441              91.500129              76.255814    0            1   0               1         0         0                          0                        0   

   Electronic check  Mailed check  
0                 1             0  
1                 0             1  
2                 0             1  
3                 0             0  
4                 1             0

In [0]:
df1=df.drop(['customerID','gender','Churn'],axis=1)

In [0]:
#Scaling is performed
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
scaler = preprocessing.RobustScaler()
scaled = scaler.fit_transform(df1)

In [0]:
scaled=pd.DataFrame(scaled,columns=df1.columns)

In [81]:
scaled.head()

SeniorCitizen  Partner  Dependents    tenure  PhoneService  MultipleLines  OnlineSecurity  OnlineBackup  DeviceProtection  TechSupport  StreamingTV  StreamingMovies  MonthlyCharges  avg_monthly_charge  avg_monthly_charge_P  avg_monthly_charge_c  avg_monthly_charge_d  avg_monthly_charge_t  avg_monthly_charge_sc  avg_monthly_charge_is  avg_monthly_charge_pm  DSL  Fiber optic   No  Month-to-month  One year  Two year  Bank transfer (automatic)  Credit card (automatic)  Electronic check  \
0            0.0      1.0         0.0 -0.608696          -1.0            0.0             0.0           1.0               0.0          0.0          0.0              0.0       -0.745170                 1.0                   1.0                   0.0                  -1.0             -1.066164                    0.0                    0.0               0.930182  1.0          0.0  0.0             0.0       0.0       0.0                        0.0                      0.0               1.0   
1            0.0      0.0         0.0  0.108696           0.0            0.0             1.0           0.0               1.0          0.0          0.0              0.0       -0.246550                 0.0                   0.0                  -1.0                   0.0              0.486698                    0.0                    0.0              -2.388850  1.0          0.0  0.0            -1.0       1.0       0.0                        0.0                      0.0               0.0   
2            0.0      0.0         0.0 -0.586957           0.0            0.0             1.0           1.0               0.0          0.0          0.0              0.0       -0.303588                 0.0                   0.0                   0.0                   0.0             -0.521454                    0.0                    0.0              -2.388850  1.0          0.0  0.0             0.0       0.0       0.0                        0.0                      0.0               0.0   
3            0.0      0.0         0.0  0.347826          -1.0            0.0             1.0           0.0               1.0          1.0          0.0              0.0       -0.516099                 0.0                   0.0                  -1.0                   0.0              0.616485                    0.0                    0.0               0.000000  1.0          0.0  0.0            -1.0       1.0       0.0                        1.0                      0.0               0.0   
4            0.0      0.0         0.0 -0.586957           0.0            0.0             0.0           0.0               0.0          0.0          0.0              0.0        0.006440                 1.0                   0.0                   0.0                  -1.0             -0.521454                    0.0                    1.0               0.930182  0.0          1.0  0.0             0.0       0.0       0.0                        0.0                      0.0               1.0   

   Mailed check  
0           0.0  
1           1.0  
2           1.0  
3           0.0  
4           0.0

In [0]:
scaled['Churn']=df['Churn']
target_col=['Churn']

In [0]:
# Correlation matrix is formed
corr_matrix=df.corr().abs()
upper=corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))
to_drop=[column for column in upper.columns if any(upper[column]>0.75)]

In [84]:
to_drop

['avg_monthly_charge_P',
 'avg_monthly_charge_d',
 'avg_monthly_charge_t',
 'avg_monthly_charge_sc',
 'avg_monthly_charge_is',
 'Fiber optic',
 'No',
 'Month-to-month',
 'Two year',
 'Mailed check']

In [0]:
scaled=scaled.drop(to_drop,axis=1)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, scorer
from sklearn.metrics import f1_score
import statsmodels.api as sm
from sklearn.metrics import precision_score, recall_score
from yellowbrick.classifier import DiscriminationThreshold

#splitting train and test
train, test = train_test_split(scaled, test_size=0.2, random_state=17)
train_X = train.drop('Churn', axis=1)
train_Y = train['Churn']
test_X = test.drop('Churn', axis=1)
test_Y = test['Churn']

In [0]:
# Importing the Keras libraries and packages 
import keras 
from keras.models import Sequential 
from keras.layers import Dense

In [0]:
clf_=Sequential()

In [0]:
clf_.add(Dense(units = 6, kernel_initializer = 'uniform', 
activation = 'relu', input_dim = 21))

In [0]:
clf_.add(Dense(units = 6, kernel_initializer = 'uniform', 
activation = 'relu'))

In [0]:
clf_.add(Dense(units = 1, kernel_initializer = 'uniform', 
activation = 'sigmoid'))

In [0]:
# Compiling Neural Network 
clf_.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [93]:
clf_.fit(train_X,train_Y, batch_size = 10, epochs = 50)




Epoch 1/50





5634/5634 [==============================] - 2s 284us/step - loss: 0.5115 - acc: 0.7350
Epoch 2/50
5634/5634 [==============================] - 1s 115us/step - loss: 0.4417 - acc: 0.7657
Epoch 3/50
5634/5634 [==============================] - 1s 120us/step - loss: 0.4351 - acc: 0.7891
Epoch 4/50
5634/5634 [==============================] - 1s 122us/step - loss: 0.4303 - acc: 0.7955
Epoch 5/50
5634/5634 [==============================] - 1s 120us/step - loss: 0.4279 - acc: 0.7946
Epoch 6/50
5634/5634 [==============================] - 1s 125us/step - loss: 0.4259 - acc: 0.7978
Epoch 7/50
5634/5634 [==============================] - 1s 116us/step - loss: 0.4239 - acc: 0.7994
Epoch 8/50
5634/5634 [==============================] - 1s 119us/step - loss: 0.4227 - acc: 0.8007
Epoch 9/50
5634/5634 [==============================] - 1s 121us/step - loss: 0.4210 - acc: 0.8007
Epoch 10/50
5634/5634 [==============================] - 1s 119us/step - loss: 0.4206 - acc: 0.8021
E

In [0]:
# Predicting the Test set results
y_pred = clf_.predict(test_X)
y_pred = (y_pred > 0.5)

In [99]:
cm = confusion_matrix(test_Y, y_pred)
print (cm)

[[930 100]
 [187 192]]


In [100]:
print(classification_report(test_Y,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.90      0.87      1030
           1       0.66      0.51      0.57       379

    accuracy                           0.80      1409
   macro avg       0.75      0.70      0.72      1409
weighted avg       0.79      0.80      0.79      1409



In [104]:
input_data = np.array([-1, 2])
weights = {
   'node_0': np.array([3, 5]),
   'node_1': np.array([1, 5]),
   'output': np.array([2, -1])
}
node_0_input = (input_data * weights['node_0']).sum()
node_0_output = np.tanh(node_0_input)
node_1_input = (input_data * weights['node_1']).sum()
node_1_output = np.tanh(node_1_input)
hidden_layer_output = np.array(node_0_output, node_1_output)
output =(hidden_layer_output * weights['output']).sum()
print(output)

def relu(input):
   '''Define your relu activation function here'''
   # Calculate the value for the output of the relu function: output
   output = max(input,0)
      # Return the value just calculated
   return(output)
# Calculate node 0 value: node_0_output
node_0_input = (input_data * weights['node_0']).sum()
node_0_output = relu(node_0_input)

# Calculate node 1 value: node_1_output
node_1_input = (input_data * weights['node_1']).sum()
node_1_output = relu(node_1_input)

# Put node values into array: hidden_layer_outputs
hidden_layer_outputs = np.array([node_0_output, node_1_output])

# Calculate model output (do not apply relu)
model_output = (hidden_layer_outputs * weights['output']).sum()
print(model_output)# Print model output

0.9999983369439447
5


In [105]:
# Define predict_with_network()
def predict_with_network(input_data_row, weights):
   # Calculate node 0 value
   node_0_input = (input_data_row * weights['node_0']).sum()
   node_0_output = relu(node_0_input)
   
   # Calculate node 1 value
   node_1_input = (input_data_row * weights['node_1']).sum()
   node_1_output = relu(node_1_input)
   
   # Put node values into array: hidden_layer_outputs
   hidden_layer_outputs = np.array([node_0_output, node_1_output])
   
   # Calculate model output
   input_to_final_layer = (hidden_layer_outputs*weights['output']).sum()
   model_output = relu(input_to_final_layer)
# Return model output
   return(model_output)

# Create empty list to store prediction results
results = []
for input_data_row in input_data:
   # Append prediction to results
   results.append(predict_with_network(input_data_row, weights))
print(results)# Print results

[0, 20]


In [107]:
input_data = np.array([3, 5])
weights = {
   'node_0_0': np.array([3, 4]),
   'node_0_1': np.array([4, -5]),
   'node_1_0': np.array([-1, 1]),
   'node_1_1': np.array([2, 2]),
   'output': np.array([2, 7])
}
def predict_with_network(input_data):
   # Calculate node 0 in the first hidden layer
   node_0_0_input = (input_data * weights['node_0_0']).sum()
   node_0_0_output = relu(node_0_0_input)
   
   # Calculate node 1 in the first hidden layer
   node_0_1_input = (input_data*weights['node_0_1']).sum()
   node_0_1_output = relu(node_0_1_input)
   
   # Put node values into array: hidden_0_outputs
   hidden_0_outputs = np.array([node_0_0_output, node_0_1_output])
   
   # Calculate node 0 in the second hidden layer
   node_1_0_input = (hidden_0_outputs*weights['node_1_0']).sum()
   node_1_0_output = relu(node_1_0_input)
   
   # Calculate node 1 in the second hidden layer
   node_1_1_input = (hidden_0_outputs*weights['node_1_1']).sum()
   node_1_1_output = relu(node_1_1_input)
   
   # Put node values into array: hidden_1_outputs
   hidden_1_outputs = np.array([node_1_0_output, node_1_1_output])
   
   # Calculate model output: model_output
   model_output = (hidden_1_outputs*weights['output']).sum()
      # Return model_output
   return(model_output)
output = predict_with_network(input_data)
print(output)

406
